# Non-invasive on-site method for thickness measurement of transparent ceramic glazes based on Depth from Focus

Here is the Jupyter Notebook used for the measurement.
In a first time, you need to use main_ceramics.m on MATLAB to generate a depthmap. This depthmap is exported to a pkl format (depthmap_python.pkl) and then can be study here in order to measure the depth of the glaze. 

Warning : don't forgot to multiply the obtained depth by the refractive index of the glaze.

## Import

Run it two time to get the interactive plot.

In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import pandas as pd 

from sklearn.cluster import KMeans

from mpl_toolkits.mplot3d import Axes3D

# Fonctions de visualisation
%matplotlib notebook

## Input for the depth measurement 

In [ ]:
filename = "D://Depth From Focus//DepthFromFocus//2021.07.01_010//"
ROI = [[2215, 1206], [4084, 2973]]

## Plot the all in focus image and the depthmap 

In [ ]:
img = mpimg.imread(filename+'//restack.jpg')
print(np.shape(img))
plt.figure(figsize=(9,7))
plt.imshow(img[ROI[0][1]:ROI[1][1], ROI[0][0]:ROI[1][0], :])
plt.axis('off')

with open(filename+'//depthmap_python.pkl', 'rb') as fin :
        a = pickle.load(fin) 

plt.figure(figsize=(9,7))
plt.imshow(a, 'gray')
plt.axis('off')
np.shape(a)

## Plot the depth histogram

In [ ]:
data = a.reshape(len(a)*len(a[0]), 1)

# First histogram to get the range
plt.figure(figsize=(10,7))
plt.hist(data, bins = 1000, color = 'yellow',
            edgecolor = 'red')
plt.xlabel('Depth (in µm)')
plt.ylabel('Numbers')
plt.title('Histogram of pixels depth')

In [ ]:
#################################################
# After the previous plot, determine the minimum
# and the maximum detph to see a better histogram
#################################################
mini = -1100
maxi = -750
#################################################

plt.figure(figsize=(10,7))
plt.hist(data, range = (mini, maxi), bins = 2000, color = 'yellow',
            edgecolor = 'red')
plt.xlabel('Depth (in µm)')
plt.ylabel('Number')
plt.title('Histogram of pixel depth on the depth map')


## Interactive plot for a better understanding of depth

Here you can use an interactive plot. Clicking on a depth in the histogram highlights the corresponding depths in red on the depthmap.

In [ ]:
######################################################
# Input val as the precision of the highlighted part
# in red : bigger is val, more values around the depth 
# are highlighted
######################################################
val = 20
######################################################

class Cursor:
    def __init__(self, fig, ax, ax2, img, data, mini, maxi, val):
        self.fig = fig
        self.ax = ax
        self.ax2 = ax2
        self.ly = ax.axvline(color='k', x = (mini+maxi)/2)  # the vert line
        self.data = data
        self.img = img
        self.value = val

        # text location in axes coords
        self.txt = ax.text(0.7, 0.9, '', transform=ax.transAxes)

    def mouse_move(self, event):
        if not event.inaxes:
            return

        x = event.xdata
        # update the line positions
        self.ly.set_xdata(x)

        self.txt.set_text('value=%1.2f' % (x))
        img2 = np.zeros((np.shape(self.img)[0], np.shape(self.img)[1], np.shape(self.img)[2]))
        img2[abs(self.data-x) <= self.value] = [1, 0, 0]
        img2[abs(self.data-x) > self.value] = self.img[abs(self.data-x) > self.value]
        self.ax2 = plt.imshow(img2)
        plt.axis('off')
        
        self.ax.figure.canvas.draw()

# Final image
with open(filename+'//depthmap_python.pkl', 'rb') as fin :
        a = pickle.load(fin) 
        
b = np.ones((np.shape(a)[0], np.shape(a)[1], 3))
for x in range(np.shape(a)[0]) :
    for y in range(np.shape(a)[1]) :
        b[x,y] =[a[x,y], a[x,y], a[x,y]]

img = (b - np.min(b))/(np.max(b)- np.min(b))
 
# Histogram    
fig = plt.figure(figsize=(10,9))
ax = fig.add_subplot(211)
ax.hist(data, range = (mini, maxi), bins = 1000, color = 'red',
            edgecolor = 'red')

ax2 = fig.add_subplot(212)


# Set useblit=True on most backends for enhanced performance.
#cursor = Cursor(ax, useblit=True, horizOn=False, color='red', linewidth=2)
cursor = Cursor(fig, ax, ax2, img, a, mini, maxi, val)
fig.canvas.mpl_connect('button_press_event', cursor.mouse_move)




plt.show()